In [21]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
url='http://deis.minsal.cl/wp-content/uploads/2020/08/DEFUNCIONES_FUENTE_DEIS_2016_2020_20082020.rar'

In [5]:
req = requests.get(url)

In [6]:
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
open(path+'DEIS.rar', 'wb').write(req.content)

4595747

In [8]:
if not os.path.exists(path+'DEIS'+timestamp):
    os.makedirs(path+'DEIS'+timestamp)
    patoolib.extract_archive(path+'DEIS.rar', outdir=path+'DEIS'+timestamp)

patool: Extracting /data/ETLcache/DEIS.rar ...
patool: running /usr/bin/rar x -- /data/ETLcache/DEIS.rar
patool:     with cwd='/data/ETLcache/DEIS_29082020_232147'
patool: ... /data/ETLcache/DEIS.rar extracted to `/data/ETLcache/DEIS_29082020_232147'.


In [9]:
for f in os.listdir(path+'DEIS'+timestamp):
    if '.csv' in f:
        deis=pd.read_csv(path+'DEIS'+timestamp+'/'+f,sep=';',encoding='latin1',header=None)
    if '.xlsx' in f:
        headers=pd.read_excel(path+'DEIS'+timestamp+'/'+f,header=3,index_col=1)

In [10]:
deis.columns=headers['Nombre de la variable']

In [13]:
deis['Metropolitana']=0
deis.loc[deis['GLOSA_REG_RES']=='Metropolitana de Santiago','Metropolitana']=1
deis['Metropolitana']=deis['Metropolitana'].replace({0:'Resto de Chile',1:'Región Metropolitana'})

In [14]:
deis['FECHA_DEF']=pd.to_datetime(deis['FECHA_DEF'])
deis['FECHA_DEF']=deis['FECHA_DEF'].dt.to_pydatetime()

In [37]:
deis['FECHA_STR']=deis['FECHA_DEF'].dt.strftime('%d-%m-%Y')

In [32]:
deis['COVID']=deis['DIAG1'].replace({'U071':'Covid PCR+','U072':'Covid Sospechoso'})
deis.loc[~deis['DIAG1'].isin(['U071','U072']),'COVID']=np.nan

In [33]:
deis['COVID'].value_counts()

Covid PCR+          10792
Covid Sospechoso     4146
Name: COVID, dtype: int64

In [38]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS exists, will not be created


In [39]:
schema='DEIS'
for y in deis['ANO_DEF'].unique():
    name='DEIS_'+str(y).split('.')[0]
    df=deis[deis['ANO_DEF']==y]
    print("creating table "+name+' ,schema: '+schema)
    df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    print("saving table"+path+name+timestamp+'.csv in cache')
    df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table DEIS_2020 ,schema: DEIS
saving table/data/ETLcache/DEIS_2020_29082020_232147.csv in cache
creating table DEIS_nan ,schema: DEIS
saving table/data/ETLcache/DEIS_nan_29082020_232147.csv in cache
creating table DEIS_2019 ,schema: DEIS
saving table/data/ETLcache/DEIS_2019_29082020_232147.csv in cache
creating table DEIS_2018 ,schema: DEIS
saving table/data/ETLcache/DEIS_2018_29082020_232147.csv in cache
creating table DEIS_2017 ,schema: DEIS
saving table/data/ETLcache/DEIS_2017_29082020_232147.csv in cache
creating table DEIS_2016 ,schema: DEIS
saving table/data/ETLcache/DEIS_2016_29082020_232147.csv in cache


In [40]:
deis

Nombre de la variable,ANO_DEF,FECHA_DEF,GLOSA_SEXO,EDAD_TIPO,EDAD_CANT,CODIGO_COMUNA_RESIDENCIA,GLOSA_COMUNA_RESIDENCIA,GLOSA_REG_RES,DIAG1,CAPITULO_DIAG1,...,GLOSA_CAPITULO_DIAG2,CODIGO_GRUPO_DIAG2,GLOSA_GRUPO_DIAG2,CODIGO_CATEGORIA_DIAG2,GLOSA_CATEGORIA_DIAG2,CODIGO_SUBCATEGORIA_DIAG2,GLOSA_SUBCATEGORIA_DIAG2,Metropolitana,COVID,FECHA_STR
0,2020.0,2020-04-19,Mujer,1.0,102.0,13132.0,Vitacura,Metropolitana de Santiago,I678,I00-I99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,19-04-2020
1,2020.0,2020-05-01,Mujer,1.0,99.0,6101.0,Rancagua,Del Libertador B. O'Higgins,F03X,F00-F99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Resto de Chile,NaN,01-05-2020
2,2020.0,2020-07-03,Hombre,1.0,95.0,13129.0,San Joaquín,Metropolitana de Santiago,U071,U00-U99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,Covid PCR+,03-07-2020
3,2020.0,2020-08-17,Mujer,1.0,101.0,4101.0,La Serena,De Coquimbo,F03X,F00-F99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Resto de Chile,NaN,17-08-2020
4,2020.0,2020-05-05,Mujer,1.0,90.0,13120.0,Ñuñoa,Metropolitana de Santiago,F03X,F00-F99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,05-05-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507748,2016.0,2016-02-08,Mujer,1.0,83.0,13110.0,La Florida,Metropolitana de Santiago,B349,A00-B99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,08-02-2016
507749,2016.0,2016-02-06,Hombre,1.0,64.0,13122.0,Peñalolén,Metropolitana de Santiago,I619,I00-I99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,06-02-2016
507750,2016.0,2016-02-07,Mujer,1.0,58.0,13110.0,La Florida,Metropolitana de Santiago,C920,C00-D48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,07-02-2016
507751,2016.0,2016-02-08,Hombre,1.0,72.0,13111.0,La Granja,Metropolitana de Santiago,K703,K00-K93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,08-02-2016


In [41]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

0